In [ ]:
import face_recognition

In [ ]:
import cv2
import mediapipe as mp

# Initialize Mediapipe Face Detection and Drawing utilities
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

def detect_faces(image_path):
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        print("Image not found. Check the file path.")
        return None

    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
        # Convert the image to RGB
        results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # If faces are detected, draw the bounding boxes
        if results.detections:
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection)

        # Return the processed image and detection results
        return image, results.detections

# Specify the image path
image_path = "/home/harshit/Desktop/hi/influencer_analysis/notebooks/frames/frame_150.jpg"  # Replace with your image file path

# Detect faces and get the processed image
output_image, detections = detect_faces(image_path)

# Print the number of detected faces
if detections:
    print(f"Detected {len(detections)} faces.")
else:
    print("No faces detected.")

# Display the image with bounding boxes
if output_image is not None:
    cv2.imshow("Detected Faces", output_image)
    cv2.waitKey(0)  # Wait for a key press to close the window
    cv2.destroyAllWindows()


libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open zink: /usr/lib/dri/zink_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)

libEGL warning: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: 

No faces detected.


In [ ]:
from deepface import DeepFace

def verify_faces(img1_path, img2_path):
    result = DeepFace.verify(img1_path, img2_path)
    return result["verified"]

img1_path = "images.jpeg"  # Replace with your image paths
img2_path = "316605.3.webp"
is_same_person = verify_faces(img1_path, img2_path)
print(f"Are these the same person? {is_same_person}")


In [ ]:
import os
import requests
from tqdm import tqdm

def download_video(url, output_dir):
    filename = url.split("/")[-1]
    file_path = os.path.join(output_dir, filename)
    print(f"Attempting to save to: {file_path}")
    
    if os.path.exists(file_path):
        print(f"{filename} already downloaded.")
        return file_path

    response = requests.get(url, stream=True)
    if response.status_code == 200:
        total_size = int(response.headers.get("content-length", 0))
        with open(file_path, "wb") as f, tqdm(
            desc=f"Downloading {filename}",
            total=total_size,
            unit="B",
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for chunk in response.iter_content(chunk_size=1024):
                f.write(chunk)
                bar.update(len(chunk))
        if os.path.exists(file_path):
            print(f"Downloaded and saved: {file_path}")
        else:
            print(f"Failed to save file: {file_path}")
        return file_path
    else:
        print(f"Failed to download {url}, HTTP status: {response.status_code}")
        return None

def download_videos_from_urls(url_list, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for url in url_list:
        download_video(url, output_dir)

# Example usage
video_urls = [
    "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-997580728807604",
]  # Add more URLs here

videos_dir = "influencer_analysis/videos"
download_videos_from_urls(video_urls, videos_dir)


In [4]:
import requests
import os

def download_video(url, output_dir="./videos"):
    os.makedirs(output_dir, exist_ok=True)
    filename = url.split("/")[-1]  # Use the last part of the URL as the filename
    filepath = os.path.join(output_dir, filename)
    
    if os.path.exists(filepath):
        print(f"Video already exists: {filepath}")
        return filepath
    
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(filepath, "wb") as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
        print(f"Downloaded video: {filepath}")
        return filepath
    else:
        print(f"Failed to download video from {url}")
        return None

# Example usage
video_url = "https://fgimagestorage.blob.core.windows.net/facebook-assets/hd-992064161877405"
video_path = download_video(video_url)


Video already exists: ./videos/hd-992064161877405


In [5]:
import cv2

def extract_frames(video_path, output_dir="./frames", frame_rate=1):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file: {video_path}")
        return
    
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_interval = int(fps / frame_rate)
    success, frame = cap.read()
    count = 0

    os.makedirs(output_dir, exist_ok=True)

    while success:
        if count % frame_interval == 0:
            frame_filename = os.path.join(output_dir, f"frame_{count}.jpg")
            cv2.imwrite(frame_filename, frame)
        success, frame = cap.read()
        count += 1
    cap.release()
    print(f"Frames extracted to: {output_dir}")

# Example usage
frames_dir = "./frames"
extract_frames(video_path, frames_dir)


Frames extracted to: ./frames


In [1]:
import cv2
import mediapipe as mp
import os
from collections import Counter
from deepface import DeepFace
import numpy as np

# Initialize Mediapipe face detection
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

def are_faces_similar(embedding1, embedding2, threshold=0.9):
    """Check if two face embeddings are similar based on a threshold."""
    distance = np.linalg.norm(embedding1 - embedding2)  # Euclidean distance
    return distance < threshold

def detect_faces(frame_path):
    """Detect faces in a single frame and return cropped faces with bounding boxes."""
    image = cv2.imread(frame_path)
    if image is None:
        print(f"Error: Could not read image from {frame_path}")
        return [], image

    cropped_faces = []
    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.9) as face_detection:
        results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.detections:
            for detection in results.detections:
                bbox = detection.location_data.relative_bounding_box
                h, w, _ = image.shape
                x_min = int(bbox.xmin * w)
                y_min = int(bbox.ymin * h)
                box_width = int(bbox.width * w)
                box_height = int(bbox.height * h)

                # Ensure the bounding box is within image bounds
                x_min = max(0, x_min)
                y_min = max(0, y_min)
                x_max = min(w, x_min + box_width)
                y_max = min(h, y_min + box_height)

                # Crop the detected face
                cropped_face = image[y_min:y_max, x_min:x_max]
                cropped_faces.append((cropped_face, (x_min, y_min, x_max, y_max)))

    return cropped_faces, image

def count_faces_in_video(frames_dir):
    """Count faces across all frames in a directory, grouping similar faces."""
    if not os.path.exists(frames_dir):
        print(f"Error: Directory {frames_dir} does not exist.")
        return Counter(), {}

    face_counter = Counter()
    face_to_frame_map = {}
    known_embeddings = []  # Store unique face embeddings
    known_face_ids = []    # Map embeddings to unique face IDs

    for frame_file in os.listdir(frames_dir):
        frame_path = os.path.join(frames_dir, frame_file)
        if not frame_file.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue  # Skip non-image files

        faces, image = detect_faces(frame_path)
        if faces:
            for cropped_face, bbox in faces:
                try:
                    # Step 1: Extract face embedding
                    embedding = DeepFace.represent(img_path=cropped_face, model_name="VGG-Face")[0]["embedding"]

                    # Step 2: Check if this embedding matches any known embedding
                    match_found = False
                    for i, known_embedding in enumerate(known_embeddings):
                        if are_faces_similar(embedding, known_embedding):
                            face_id = known_face_ids[i]
                            face_counter[face_id] += 1
                            match_found = True
                            break

                    # Step 3: If no match found, assign a new ID
                    if not match_found:
                        face_id = len(known_embeddings) + 1  # Generate a new face ID
                        known_embeddings.append(embedding)
                        known_face_ids.append(face_id)
                        face_counter[face_id] += 1
                        # Store the first occurrence of this face for visualization
                        if face_id not in face_to_frame_map:
                            face_to_frame_map[face_id] = (image, (cropped_face, bbox))
                except Exception as e:
                    print(f"Error processing face: {e}")

    return face_counter, face_to_frame_map

def display_face(image, face):
    """Draw and display a detected face on the image."""
    cropped_face, bbox = face
    x_min, y_min, x_max, y_max = bbox

    # Draw a rectangle around the face
    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

    # Display the image
    cv2.imshow("Most Frequent Face", image)
    cv2.imshow("Cropped Face", cropped_face)
    cv2.waitKey(0)
    cv2.destroyAllWindows()



# Example usage
frames_dir = "./frames"  # Replace with your directory of frames
face_counts, face_to_frame_map = count_faces_in_video(frames_dir)
most_frequent_face_id=0

if face_counts:
    most_frequent_face_id = face_counts.most_common(1)[0][0]
    print(f"Most frequent face ID: {most_frequent_face_id} with {face_counts[most_frequent_face_id]} occurrences")
    # Display the most frequent face
    image, face = face_to_frame_map[most_frequent_face_id]
    display_face(image, face)
else:
    print("No faces detected in the video.")


2024-11-30 03:33:25.812730: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-30 03:33:25.820093: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-30 03:33:25.976588: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-30 03:33:25.979606: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-30 03:33:28.460912: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

No faces detected in the video.


In [2]:
import numpy as np
import os

def save_face_embedding(face_id, face_embedding, embedding_dir="./embeddings"):
    """
    Save the face embedding or unique identifier to a file.
    :param face_id: Unique ID of the face.
    :param face_embedding: Data to save (e.g., an array or identifier).
    :param embedding_dir: Directory to save the embeddings.
    :return: Path to the saved embedding file.
    """
    os.makedirs(embedding_dir, exist_ok=True)  # Create the directory if it doesn't exist
    embedding_path = os.path.join(embedding_dir, f"face_{face_id}.npy")
    np.save(embedding_path, face_embedding)  # Save the face embedding
    print(f"Saved face embedding: {embedding_path}")
    return embedding_path

# Example usage

if(most_frequent_face_id!=0):
    embedding_path = save_face_embedding(most_frequent_face_id, str(face))

In [3]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def are_embeddings_similar(file1, file2, threshold=0.2):
    """
    Compare two .npy embedding files and determine if they are similar.
    
    :param file1: Path to the first .npy file.
    :param file2: Path to the second .npy file.
    :param threshold: Cosine similarity threshold for determining similarity.
    :return: True if embeddings are similar, False otherwise.
    """
    # Load embeddings from .npy files
    embedding1 = np.load(file1)
    embedding2 = np.load(file2)

    # Ensure embeddings are 2D for cosine similarity calculation
    embedding1 = embedding1.reshape(1, -1)
    embedding2 = embedding2.reshape(1, -1)

    # Compute cosine similarity
    similarity = cosine_similarity(embedding1, embedding2)[0][0]

    # Compare similarity to the threshold
    print(f"Cosine Similarity: {similarity:.4f}")
    return similarity >= threshold

# Example usage
file1 = "/home/harshit/Desktop/hi/influencer_analysis/notebooks/embeddings/hd-604041931565137/face_1.npy"  # Replace with your actual path
file2 = "/home/harshit/Desktop/hi/influencer_analysis/notebooks/embeddings/hd-605272664726548/face_1.npy"  # Replace with your actual path

if are_embeddings_similar(file1, file2):
    print("The embeddings are similar.")
else:
    print("The embeddings are not similar.")


Cosine Similarity: 0.1820
The embeddings are not similar.
